[View in Colaboratory](https://colab.research.google.com/github/dsaha78/eip/blob/master/Yolo_Basic.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 3
l = 10
num_filter = 20

Using TensorFlow backend.


In [2]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 14s 0us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer_1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer_1  = BatchNormalization(name='norm_1')(layer_1)
layer_1  = LeakyReLU(alpha=0.1)(layer_1)
layer_1  = MaxPooling2D(pool_size=(2, 2))(layer_1)

# Layer 2
layer_2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer_1)
layer_2 = BatchNormalization(name='norm_2')(layer_2)
layer_2 = LeakyReLU(alpha=0.1)(layer_2)
layer_2 = MaxPooling2D(pool_size=(2, 2))(layer_2)

# Layer 3
layer_3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer_2)
layer_3 = BatchNormalization(name='norm_3')(layer_3)
layer_3 = LeakyReLU(alpha=0.1)(layer_3)

# Layer 4
layer_4 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer_3)
layer_4 = BatchNormalization(name='norm_4')(layer_4)
layer_4 = LeakyReLU(alpha=0.1)(layer_4)

# Layer 5
layer_5 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer_4)
layer_5 = BatchNormalization(name='norm_5')(layer_5)
layer_5 = LeakyReLU(alpha=0.1)(layer_5)
layer_5 = MaxPooling2D(pool_size=(2, 2))(layer_5)

# Layer 6
layer_6 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer_5)
layer_6 = BatchNormalization(name='norm_6')(layer_6)
layer_6 = LeakyReLU(alpha=0.1)(layer_6)

# Layer 7
layer_7 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer_6)
layer_7 = BatchNormalization(name='norm_7')(layer_7)
layer_7 = LeakyReLU(alpha=0.1)(layer_7)

# Layer 8
layer_8 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer_7)
layer_8 = BatchNormalization(name='norm_8')(layer_8)
layer_8 = LeakyReLU(alpha=0.1)(layer_8)
layer_8 = MaxPooling2D(pool_size=(2, 2))(layer_8)

# Layer 9
layer_9 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer_8)
layer_9 = BatchNormalization(name='norm_9')(layer_9)
layer_9 = LeakyReLU(alpha=0.1)(layer_9)

# Layer 10
layer_10 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer_9)
layer_10 = BatchNormalization(name='norm_10')(layer_10)
layer_10 = LeakyReLU(alpha=0.1)(layer_10)

# Layer 11
layer_11 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer_10)
layer_11 = BatchNormalization(name='norm_11')(layer_11)
layer_11 = LeakyReLU(alpha=0.1)(layer_11)

# Layer 12
layer_12 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer_11)
layer_12 = BatchNormalization(name='norm_12')(layer_12)
layer_12 = LeakyReLU(alpha=0.1)(layer_12)

skip_connection = layer_12

# Layer 13
layer_13 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer_12)
layer_13 = BatchNormalization(name='norm_13')(layer_13)
layer_13 = LeakyReLU(alpha=0.1)(layer_13)

#skip_connection = layer_13

layer_13 = MaxPooling2D(pool_size=(2, 2))(layer_13)

# Layer 14
layer_14 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer_13)
layer_14 = BatchNormalization(name='norm_14')(layer_14)
layer_14 = LeakyReLU(alpha=0.1)(layer_14)

# Layer 15
layer_15 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer_14)
layer_15 = BatchNormalization(name='norm_15')(layer_15)
layer_15 = LeakyReLU(alpha=0.1)(layer_15)

# Layer 16
layer_16 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer_15)
layer_16 = BatchNormalization(name='norm_16')(layer_16)
layer_16 = LeakyReLU(alpha=0.1)(layer_16)

# Layer 17
layer_17 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer_16)
layer_17 = BatchNormalization(name='norm_17')(layer_17)
layer_17 = LeakyReLU(alpha=0.1)(layer_17)

# Layer 18
layer_18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer_17)
layer_18 = BatchNormalization(name='norm_18')(layer_18)
layer_18 = LeakyReLU(alpha=0.1)(layer_18)

# Layer 19
layer_19 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer_18)
layer_19 = BatchNormalization(name='norm_19')(layer_19)
layer_19 = LeakyReLU(alpha=0.1)(layer_19)

# Layer 20
layer_20 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(layer_19)
layer_20 = BatchNormalization(name='norm_20')(layer_20)
layer_20 = LeakyReLU(alpha=0.1)(layer_20)

# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

layer_21 = concatenate([skip_connection, layer_20])

# Layer 22
layer_22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(layer_21)
layer_22 = BatchNormalization(name='norm_22')(layer_22)
layer_22 = LeakyReLU(alpha=0.1)(layer_22)

# Layer 23
layer_23 = Flatten()(layer_22)

output = Dense(num_classes, activation='softmax')(layer_23)



In [23]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_12[0][0]                   
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_194 (LeakyReLU)     (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
max_poolin

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 102s 2ms/step - loss: 1.4506 - acc: 0.4706 - val_loss: 3.3633 - val_acc: 0.2862
Epoch 2/3
50000/50000 [==============================] - 94s 2ms/step - loss: 1.0012 - acc: 0.6439 - val_loss: 1.3219 - val_acc: 0.5848
Epoch 3/3
50000/50000 [==============================] - 94s 2ms/step - loss: 0.7975 - acc: 0.7195 - val_loss: 0.9935 - val_acc: 0.6625


In [26]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 10s 1ms/step
Test loss: 0.993465172958374
Test accuracy: 0.6625
Saved the model to disk


In [28]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')

Saved the model to disk


FileNotFoundError: ignored